In [1]:
!pip install transformers datasets torch

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   ----- ---------------------------------- 1.3/10.0 MB 6.7 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/10.0 MB 6.6 MB/s eta 0:00:02
   ---------------- ----------------------- 4.2/10.0 MB 6.6 MB/s eta 0:00:01
   --------------------- ------------------ 5.5/10.0 MB 6.6 MB/s eta 0:00:01
   --------------------------- ------------ 6.8/10.0 MB 6.5 MB/s eta 0:00:01
   ------------------------------ --------- 7.6/10.0 MB 6.5 MB/s eta 0:00:01
   ------------------------------------- -- 9.4/10.0 MB 6.5 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/203.0 MB ? eta -:--:--
   ---------------------------------------- 1.3/203.0 MB 6.1 MB/s eta 0:00:34
    --------------------------------------- 2.6/203.0 MB 6.6 MB/s eta 0:00:31
    --------------------------------------- 3.9/203.0 MB 6.5 MB/s eta 0:00:31
   - ---

In [3]:
!pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------ --------------- 1.0/1.7 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 6.2 MB/s eta 0:00:00


In [13]:
!pip install transformers[torch] accelerate

In [14]:
pip show transformers accelerate torch

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: 
---
Name: accelerate
Version: 1.1.1
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
---
Name: torch
Version: 2.5.1
Summary: Tensors and Dynamic neu

ERROR: Invalid requirement: "'accelerate": Expected package name at the start of dependency specifier
    'accelerate
    ^


In [1]:
import pandas as pd
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset, DataLoader
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load dataset
df = pd.read_csv(r"data\data_trustpilot.csv") 
df.head()

,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,...,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,5,CA,Rob Crane,2,Redirected,repeat,1,Flashbay,The company rep I worked with made my transact...,company rep worked made transaction smooth qui...,...,2024-10-23 04:17:44,2024-10-22,10,21,Evening,1,Business Day,129,late_review,0
1,5,US,Pat Anderson,1,Verified,one-time,0,Flashbay,I highly recommend using Flashbay. Immediately...,highly recommend using flashbay immediately or...,...,2024-10-16 19:34:05,2024-10-16,10,12,Business Hours,2,Business Day,0,quick_review,1
2,5,CZ,Margarita Orlova,1,Verified,one-time,0,Flashbay,I had the pleasure of working with Shelby Gibs...,pleasure working shelby gibson large order nee...,...,2024-10-17 10:27:44,2024-10-17,10,10,Business Hours,3,Business Day,7,late_review,0
3,5,US,Paola Rivas,1,Verified,one-time,0,Flashbay,I had a fantastic experience with Brian Truong...,fantastic experience brian truong attentive tr...,...,2024-10-21 22:38:50,2024-10-21,10,15,Business Hours,0,Business Day,0,quick_review,1
4,5,CA,Fiona Mckelvey Keenan,3,Not Verified,repeat,1,Flashbay,My number-one go-to for computer accessories. ...,numberone goto computer accessories rachel sup...,...,2024-10-23 04:09:05,2024-10-22,10,21,Evening,1,Business Day,103,late_review,0


In [4]:
df["rating"].value_counts()

rating
5    31372
1    16152
4     6290
3     6272
2     4324
Name: count, dtype: int64

In [5]:
# Find the size of the smallest class
smallest_class_size = df['rating'].value_counts().min()

# Separate the classes based on ratings
rating_1 = df[df['rating'] == 1]
rating_3 = df[df['rating'] == 3]
rating_4 = df[df['rating'] == 4]
rating_5 = df[df['rating'] == 5]
other_ratings = df[df['rating'] == 2]

# Downsample the majority classes (ratings 1 and 5) to the smallest class size
rating_1_downsampled = rating_1.sample(n=smallest_class_size, random_state=42)
rating_3_downsampled = rating_3.sample(n=smallest_class_size, random_state=42)
rating_4_downsampled = rating_4.sample(n=smallest_class_size, random_state=42)
rating_5_downsampled = rating_5.sample(n=smallest_class_size, random_state=42)

# Combine back the dataset
df_balanced = pd.concat([rating_1_downsampled, rating_3_downsampled, rating_4_downsampled, rating_5_downsampled, other_ratings])

# Shuffle the dataset
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# Check the new class distribution
print(df_balanced['rating'].value_counts())

rating
1    4324
4    4324
2    4324
5    4324
3    4324
Name: count, dtype: int64


In [6]:
# readjustment to make it fit in the model 
df_balanced['rating'] = df_balanced['rating'] - 1

In [7]:
df_balanced.head()

,rating,location,username,number_reviews,verification,repeat_reviewer,repeat_reviewer_encoded,company,text,text_processed,...,date_posted,local_date_posted,month_local,local_hour,time_of_day,day_of_week_posted,day_type,days_between_experience_and_post,review_time,review_time_encoded
0,0,GB,Mrs. L,4,Not Verified,repeat,1,Asda,I ordered some clothing online and returned 2 ...,ordered clothing online returned 2 items didnt...,...,2024-09-01 17:04:31,2024-09-01,9,16,Business Hours,6,Weekend,27,late_review,0
1,0,TR,Berkay,1,Verified,one-time,0,SteelSeries,Can't ship to my country and left me,cant ship country left,...,2021-06-29 23:06:07,2021-06-30,6,0,Night,2,Business Day,1,late_review,0
2,0,GB,Philippa Young,2,Not Verified,repeat,1,Asda,Very poor service. I ordered some clothes for ...,poor service ordered clothes children click co...,...,2024-09-21 09:05:08,2024-09-21,9,8,Morning,5,Weekend,0,quick_review,1
3,3,US,Deborah,2,Verified,repeat,1,Quicken,I can easily keep track of my family's expense...,easily keep track familys expenses taxes breez...,...,2023-02-25 22:05:51,2023-02-25,2,15,Business Hours,5,Weekend,0,quick_review,1
4,1,US,Lester,1,Invited,one-time,0,Quicken,Much too complicated to use compared to the ea...,much complicated use compared earlier version ...,...,2023-09-13 00:52:29,2023-09-12,9,17,Business Hours,1,Business Day,73,late_review,0


In [ ]:
# Select the column to use as input 
input_column = 'text'  

# Tokenization and Dataset Preparation
class ReviewDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        text = row[input_column]
        rating = row['rating']
        tokens = self.tokenizer(
            text, 
            max_length=self.max_length, 
            truncation=True, 
            padding='max_length', 
            return_tensors='pt'
        )
        return {
            'input_ids': tokens['input_ids'].squeeze(),
            'attention_mask': tokens['attention_mask'].squeeze(),
            'labels': torch.tensor(rating, dtype=torch.long)
        }

# Initialize tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Prepare Dataset
max_length = 128
dataset = ReviewDataset(df_balanced, tokenizer, max_length)

# Split dataset into train and test
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_balanced, test_size=0.2, random_state=42)
train_dataset = ReviewDataset(train_df, tokenizer, max_length)
test_dataset = ReviewDataset(test_df, tokenizer, max_length)

# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    learning_rate=2e-5,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Initialize Trainer
def compute_metrics(eval_pred):
    from sklearn.metrics import accuracy_score, precision_recall_fscore_support
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=1)
    accuracy = accuracy_score(labels, predictions)
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(f"Test Results: {results}")



c:\Users\Admin\.conda\envs\supply_chain\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--distilbert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dist

{'loss': 1.6069, 'grad_norm': 1.6705536842346191, 'learning_rate': 1.9938328707986433e-05, 'epoch': 0.01}


  1%|          | 20/3243 [03:18<8:30:35,  9.51s/it]

{'loss': 1.5653, 'grad_norm': 1.9342453479766846, 'learning_rate': 1.9876657415972867e-05, 'epoch': 0.02}


  1%|          | 30/3243 [04:53<8:30:43,  9.54s/it]

...

100%|██████████| 3243/3243 [10:01:07<00:00, 11.12s/it]


{'train_runtime': 36067.0495, 'train_samples_per_second': 1.439, 'train_steps_per_second': 0.09, 'train_loss': 0.8244395586873071, 'epoch': 3.0}


100%|██████████| 271/271 [13:03<00:00,  2.89s/it]


Test Results: {'eval_loss': 0.9004201292991638, 'eval_accuracy': 0.6234967622571693, 'eval_runtime': 786.1553, 'eval_samples_per_second': 5.5, 'eval_steps_per_second': 0.345, 'epoch': 3.0}


('/content/drive/MyDrive/finetuned_destilbert_unpr_tokenizer\\tokenizer_config.json',
 '/content/drive/MyDrive/finetuned_destilbert_unpr_tokenizer\\special_tokens_map.json',
 '/content/drive/MyDrive/finetuned_destilbert_unpr_tokenizer\\vocab.txt',
 '/content/drive/MyDrive/finetuned_destilbert_unpr_tokenizer\\added_tokens.json')

In [9]:
# Save the model to a directory
model_save_path = "finetuned_destilbert_unpr_model" 
trainer.save_model(model_save_path)  

# Save the tokenizer as well 
tokenizer_save_path = "finetuned_destilbert_unpr_tokenizer" 
tokenizer.save_pretrained(tokenizer_save_path)

('finetuned_destilbert_unpr_tokenizer\\tokenizer_config.json',
 'finetuned_destilbert_unpr_tokenizer\\special_tokens_map.json',
 'finetuned_destilbert_unpr_tokenizer\\vocab.txt',
 'finetuned_destilbert_unpr_tokenizer\\added_tokens.json')

In [10]:
# print classification report and confusion matrix
predictions = trainer.predict(test_dataset)

predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids


from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(true_labels, predicted_labels))
print(confusion_matrix(true_labels, predicted_labels))

100%|██████████| 271/271 [13:24<00:00,  2.97s/it]

              precision    recall  f1-score   support

           0       0.72      0.74      0.73       850
           1       0.46      0.50      0.48       875
           2       0.51      0.49      0.50       904
           3       0.65      0.52      0.58       835
           4       0.79      0.87      0.83       860

    accuracy                           0.62      4324
   macro avg       0.62      0.63      0.62      4324
weighted avg       0.62      0.62      0.62      4324

[[628 172  34  13   3]
 [188 441 222  18   6]
 [ 43 292 440 115  14]
 [ 12  45 161 438 179]
 [  1   8   8  94 749]]
